In [1]:
import numpy as np
from sklearn.decomposition import NMF
from sklearn import preprocessing
import matplotlib.pyplot as plt
min_max_scaler=preprocessing.MinMaxScaler()
import warnings
warnings.filterwarnings('ignore')
import glob
from sklearn.decomposition import FactorAnalysis
import os.path

In [2]:
maxNoOfCircle = 30

In [3]:
nodeLookUp={}
nodeIdx=-1
def getNodeId(rawId):
    global nodeIdx
    if rawId in nodeLookUp:
        nodeId = nodeLookUp[rawId]
    else:
        nodeIdx=1+nodeIdx
        nodeId = nodeIdx
        nodeLookUp[rawId] = nodeId
    return nodeId

In [4]:
noOfNodes=100
noOfTestSamples=int(0.2*noOfNodes)
testSetExpected=np.zeros((noOfNodes,noOfNodes))
testSetIdxs=np.random.randint(0,noOfNodes-1,noOfTestSamples)

def initForCircle():
    global nodeLookUp
    global nodeIdx
    nodeLookUp={}
    nodeIdx=-1
    

In [5]:
def loadCircle(circleFile):
    global noOfTestSamples
    global testSetExpected
    global testSetIdxs
    global nodeLookUp
    global noOfNodes
    circles = open(circleFile, 'r')
    for circle in circles:
        nodesInCircle = circle.split()[1:]
        for node in nodesInCircle:
            getNodeId(node)
    noOfNodes=len(nodeLookUp)
    print 'noOfNodes:',noOfNodes
    noOfTestSamples=int(noOfNodes/5)
    testSetExpected=np.zeros((noOfNodes,noOfNodes))
    testSetIdxs=np.random.randint(0,noOfNodes-1,noOfTestSamples)


In [6]:
def bigClam(utilMat,noOfCircles):
    global noOfNodes
    global nodeLookUp
    error1={}
    error2={}
    P_C={}
    ssd={}
    testSSE={}
    earlyStop=-1
    for k in range(1,maxNoOfCircle):
        P_C[k]=0
        #print 'to fit util mat with k',utilMat.shape,k
        model = NMF(n_components=k,nls_max_iter=10000,solver='cd',l1_ratio=1,alpha=0.7,init='nndsvd',tol=0.000001)
        F = model.fit_transform(utilMat)
        error1[k]=model.reconstruction_err_
        error2[k]=0
        nodeCircleProbs = np.zeros((noOfNodes,k))
        for i in range(k):
            circleModel = NMF(n_components=1,solver='cd',l1_ratio=0,alpha=1,tol=0.000001,init='nndsvd',nls_max_iter=10000,max_iter=10000)
            F_=F[:,i].reshape(len(F[:,i]),1)
            F_ab_=F_*np.transpose(F_)
            min_max_scaler.fit([0, 100])
            neg_F_ab_=-1*min_max_scaler.transform(F_ab_)*100
            exp_F_ab_=np.exp(neg_F_ab_)
            nodeCircleProbPrime = min_max_scaler.fit_transform(circleModel.fit_transform(exp_F_ab_))
            nodeCircleProb=1-nodeCircleProbPrime
            nodeCircleProbs[:,i]=nodeCircleProb.reshape(len(nodeLookUp))
            P_c=reduce(lambda y,x: np.log(np.maximum(x,0.00000000000000000000000001))+np.log(np.maximum(1-x,0.0000000000000000000001)),nodeCircleProb)
            P_C[k]=P_C[k]+P_c
            error2[k]=error2[k]+circleModel.reconstruction_err_
        reconstructedUtilMat=np.zeros((noOfNodes,noOfNodes))
        testSetActual=np.zeros((noOfNodes,noOfNodes))
        hardNodeInCircle = nodeCircleProbs > 0.5
        hardNodeInCircle = hardNodeInCircle.astype(int)
        for n1 in range(noOfNodes):
            for n2 in range(noOfNodes):
                if (n1 in testSetIdxs) and (n2 in testSetIdxs):
                    testSetActual[n1,n2] = 1 - np.prod(1-np.multiply(hardNodeInCircle[n1],hardNodeInCircle[n2]))
                    #print testSetActual[n1,n2]
        testSSE[k]=np.sum((testSetExpected-testSetActual)**2)
        for n1 in range(noOfNodes):
            for n2 in range(noOfNodes):
                reconstructedUtilMat[n1,n2]=1 - np.prod(1-np.multiply(hardNodeInCircle[n1],hardNodeInCircle[n2]))
        ssd[k]=np.sum(((utilMat/100)-reconstructedUtilMat)**2)
        #print k""",P_C[k],error1[k],error2[k]""",ssd[k],"""np.sum(nodeCircleProbs),"""testSSE[k],np.sum(testSetActual)
        #print k,ssd[k],testSSE[k],np.sum(testSetActual)
        if earlyStop==-1:
            earlyStop = ssd[k]
        elif ssd[k]>earlyStop:
            break
    print np.argmin(ssd),np.argmin(testSSE)
    return np.argmin(ssd),np.argmin(testSSE)
    

In [7]:
def loadUtilMatrix(circleFile):
    global nodeLookUp
    print 'circleFile:',circleFile
    utilMat = np.zeros((len(nodeLookUp),len(nodeLookUp)))
    circles = open(circleFile, 'r')
    np.set_printoptions(threshold=1000)
    noOfCircles=0
    for circle in circles:
        noOfCircles=noOfCircles+1
        nodesInCircle = circle.split()
        nodesInCircle = nodesInCircle[1:]
        print len(nodesInCircle)
        for node1 in nodesInCircle:
            for node2 in nodesInCircle:
                node1Idx=getNodeId(node1)
                node2Idx=getNodeId(node2)
                if (node1Idx in testSetIdxs) and (node2Idx in testSetIdxs):
                    testSetExpected[node1Idx,node2Idx]=1
                utilMat[node1Idx,node2Idx]=utilMat[node1Idx,node2Idx]+1
    print 'circles.size',noOfCircles
    return utilMat,noOfCircles


In [8]:
L_n=5
maxNoOfIterations = 10000
min_del_e=0.0001
trainCircles = '../data/circles/*.circles'
featureFile ='../data/features/features_massaged.txt'
egoNetFile ='../data/egonets/*.egonet'
featureSet=["birthday","education;classes;description","education;classes;from;id","education;classes;from;name","education;classes;id","education;classes;name","education;classes;with;id","education;classes;with;name","education;concentration;id","education;concentration;name","education;degree;id","education;degree;name","education;school;id","education;school;name","education;type","education;with;id","education;with;name","education;year;id","education;year;name","first_name","gender","hometown;id","hometown;name","id","languages;id","languages;name","last_name","locale","location","location;id","location;name","middle_name","name","political","religion","work;description","work;employer;id","work;employer;name","work;end_date","work;from;id","work;from;name","work;location;id","work;location;name","work;position;id","work;position;name","work;projects;description","work;projects;end_date","work;projects;from;id","work;projects;from;name","work;projects;id","work;projects;name","work;projects;start_date","work;projects;with;id","work;projects;with;name","work;start_date","work;with;id","work;with;name"]

In [9]:
def loadFeatures():
    nodesFeatures = open(featureFile, 'r')
    F_u=[]
    for nodeFeaturesText in nodesFeatures:
        nodeFeatures = nodeFeaturesText.split(' ')
        node = nodeFeatures[0]
        featureValue={}
        for featureValueText in nodeFeatures[1:]:
            featureValuePair=featureValueText.split(':')
            featureValue[featureValuePair[0]]=featureValuePair[1]
        f_u=[]
        for idx in range(len(featureSet)):
            f_u_i=-1
            if(featureSet[idx] in featureValue):
                f_u_i = int(featureValue[featureSet[idx]].strip())
            f_u.append(f_u_i)
        F_u.append(f_u)
            
loadFeatures()         
        

In [10]:
def loadGraphs(egoNetFile):
    uv_c={}
    for egoNetFile in glob.glob(egoNetFile):
        egoNet = open(egoNetFile, 'r')
        alter = os.path.basename(egoNetFile).split('.')[0]
        uv={}
        uv_c[alter]=uv
        for uvs in egoNet:
            u=uvs.split(' ')[0].split(':')[0].strip()
            for v in uvs[1:]:
                uv[(u,v.strip())]=1
    return uv_c
loadGraphs(egoNetFile)

{}

In [12]:
def trainCommunityMembership(utilMat,noOfCircle):
    circleModel = NMF(n_components=noOfCircle,solver='cd',l1_ratio=0,alpha=1,tol=0.000001,init='nndsvd',nls_max_iter=10000,max_iter=10000)
    circleModel.fit(utilMat)
    return circleModel
            

In [13]:
def predictNodeCommunityMembership(utilMat,circleModel):
    return circleModel.transform(utilMat)

In [14]:
def fitAndRegenerateUtilMatrix(utilMat,noOfCircle):
    circleModel = trainCommunityMembership(utilMat,noOfCircle)
    F_u_c = predictNodeCommunityMembership(utilMat,circleModel)
    return F_u_c

In [15]:
def buildUtilMatFromCircleMembership(F_u_c):
    return np.dot(nodeCircleMembership,np.transpose(nodeCircleMembership))

In [16]:
def computeFittingError(F_uv_l,F_uv_c):
    return np.sum(((F_uv_l-F_uv_c)**2)
    

SyntaxError: invalid syntax (<ipython-input-16-b6a4fee90152>, line 3)

In [17]:
def initUtilMatrix(N,k):
    return np.ones((N,N))*.25*k

In [18]:
def trainLatentAspect(X_uv,F_uv):
    fa = FactorAnalysis(n_components=k)
    fa.fit(X,F_uv)
    return fa

In [19]:
def predictNodeCommunityMembership(latentModel,X,k):
    N = X.shape[0]
    L=np.zeros((N,N,L_n))
    for u in range(N):
        for v in range(N):
            L[u,v]=latentModel.transform(generateNodePairFeatures(u,v))
    F_uv_l=initUtilMatrix(N,k)
    e_prev=0
    F_u_c = np.zeros(N,k)
    for itr in range(maxIteration):
        for u in range(N):
            for v in range(N):
                F_uv_l[u,v]=nodePairCirclesModel.fit_transform(L[u,v])
        F_u_c = fitAndRegenerateUtilMatrix(F_uv_l,k)
        F_uv_c = buildUtilMatFromCircleMembership(F_u_c)
        F_uv_c[F_uv_c<0]=0
        e = computeFittingError(F_uv_l,F_uv_c)
        if (np.absolute(e-e_prev)/e_prev<=min_del_e):
            break;
    return F_u_c,e
    

In [20]:
for circleFile in glob.glob(trainCircles):
    trainLatentAspect()
    loadCircle(circleFile)
    utilMat,noOfCircles = loadUtilMatrix(circleFile)
    k_ssd,k_testSSE=bigClam(utilMat,noOfCircles)
    print 'circle - actual,predicted',noOfCircles,k_ssd,k_testSSE

In [ ]:
%matplotlib inline

def plotHist(dictValues):
    plt.bar(range(len(dictValues)), dictValues.values())
    plt.xticks(range(len(dictValues)), dictValues.keys())
    plt.show()

#plotHist(P_C)
plotHist(ssd)
plotHist(testSSE)

